In [92]:
from shogun import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [93]:
dataframe = pd.read_csv('../data/vid-game.csv')
df = dataframe.dropna()

# df.shape

In [94]:
# training set
col_x = [col for col in df.columns
      if col != 'Genre']
X_train = df[col_x]

X_train.head()

,Name,Platform,Year_of_Release,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [95]:
# target variable
target = ['Genre']
y_train = df[target]

In [96]:
encoder = LabelEncoder()

s = (X_train.dtypes == 'object')

cat_vars = list(s[s].index)


In [97]:
X_train = X_train.drop(columns=['Name'],axis=0) #Name variable doesn't hold much info
X_train.columns

Index(['Platform', 'Year_of_Release', 'Publisher', 'NA_Sales', 'EU_Sales',
       'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')

In [98]:
#Now we have only the categorical variable
s = (X_train.dtypes == 'object')
cat_vars= list(s[s].index)

X_train[cat_vars].notna()

,Platform,Publisher,User_Score,Developer,Rating
0,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
6,True,True,True,True,True
7,True,True,True,True,True
...,...,...,...,...,...
16667,True,True,True,True,True
16677,True,True,True,True,True
16696,True,True,True,True,True
16700,True,True,True,True,True


In [99]:
#  Labeling properly the columns with object data type
le = LabelEncoder()

label_X = X_train.copy()
for col in cat_vars:
    label_X[col] = le.fit_transform(label_X[col])
y_train = le.fit_transform(y_train)  #Convert to float    


/home/psykik/anaconda3/envs/shogun/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [100]:
from time import time
from sklearn import svm

t0 = time()
skr = svm.SVC()
skr.fit(label_X, y_train)
tsk1 = time() - t0
print(tsk1)

3.892184019088745


In [101]:
#Convert features and labels suitable for shogun
label_X= label_X.to_numpy().astype(float)
y_train = y_train.astype(float)
y_train = y_train.reshape(y_train.shape[0],) #reshape to (num_samples,)

In [104]:
from shogun import *

train_features = RealFeatures(label_X.T)
labels_train = ClassificationLabels(y_train)

t0 = time()
C = 1.0
epsilon = 0.001
gauss_kernel = GaussianKernel(train_features, train_features, 15)

svm = MulticlassLibSVM(C, gauss_kernel, labels_train)
svm.set_epsilon(epsilon)

svm.train()
tsg1= time() - t0


print(tsg1)





NameError: name 'ClassificationLabels' is not defined